In [6]:
from pathlib import Path

organization = "org-NaDNa5c5In5LspIF45qqArRr" # This is mine. You should change it!

keyfile = Path.home() / '.openai' / 'api.key'
with open(keyfile) as f:
    api_key = f.readline().strip()

In [8]:
import openai
openai.organization = organization
openai.api_key = api_key
models = openai.Model.list()
models

<OpenAIObject list at 0x7f3992c0bdd0> JSON: {
  "data": [
    {
      "created": 1677532384,
      "id": "whisper-1",
      "object": "model",
      "owned_by": "openai-internal",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
          "allow_sampling": true,
          "allow_search_indices": false,
          "allow_view": true,
          "created": 1683912666,
          "group": null,
          "id": "modelperm-KlsZlfft3Gma8pI6A8rTnyjs",
          "is_blocking": false,
          "object": "model_permission",
          "organization": "*"
        }
      ],
      "root": "whisper-1"
    },
    {
      "created": 1649358449,
      "id": "babbage",
      "object": "model",
      "owned_by": "openai",
      "parent": null,
      "permission": [
        {
          "allow_create_engine": false,
          "allow_fine_tuning": false,
          "allow_logprobs": true,
      

In [16]:
gpt_model_name = "gpt-3.5-turbo"
[m for m in models.data if m.root == gpt_model_name][0]

<Model model id=gpt-3.5-turbo at 0x7f3993f48900> JSON: {
  "created": 1677610602,
  "id": "gpt-3.5-turbo",
  "object": "model",
  "owned_by": "openai",
  "parent": null,
  "permission": [
    {
      "allow_create_engine": false,
      "allow_fine_tuning": false,
      "allow_logprobs": true,
      "allow_sampling": true,
      "allow_search_indices": false,
      "allow_view": true,
      "created": 1683852530,
      "group": null,
      "id": "modelperm-C34BeeteYvuLNoa893aJRcnp",
      "is_blocking": false,
      "object": "model_permission",
      "organization": "*"
    }
  ],
  "root": "gpt-3.5-turbo"
}

In [20]:
def basic_completion(user_prompt, max_tokens=50):
    x = openai.ChatCompletion.create(
        model = gpt_model_name,
        messages = [
            {'role': 'user', 'content': user_prompt}
        ],
        max_tokens = max_tokens
    )
    return x.choices[0].message

In [23]:
basic_completion("Write a haiku about being a dungeon master")

<OpenAIObject at 0x7f3993f53880> JSON: {
  "content": "Crafting tales with care,\nFaces lit by candle's flare,\nMy world comes alive.",
  "role": "assistant"
}

In [85]:
system_prompt = '''
You are an improvisational subsystem, part of a TTRPG game master assistant utility.

The entire response must be a single valid JSON object, and must not contain any other characters.
'''

def system_completion(user_prompt):
    call = openai.ChatCompletion.create(
        model = gpt_model_name,
        messages = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt}
        ]
    )
    return call.choices[0].message

world_prompt = '''
Create a standard fantasy world with the following structure:

name: ''
regions: ''
    name: ''
    description: ''
    pointsOfInterest:
        name: ''
        description: ''
    settlements:
        name: ''
        description: ''
        size: <VILLAGE|TOWN|CITY>
factions:
    name: ''
    description: ''
    locations:
        settlement: <reference>
        presence: <NONE|LIMITED|ESTABLISHED>

All settlements must be unique. All factions' locations must reference a settlement that exists.

There should be 3-4 regions, each with 1-2 points of interest and 3-6 settlements. 

There should be 2-3 major factions, and 5-10 minor factions. 
'''

def create_world():
    return system_completion(world_prompt)


In [31]:
world = create_world()
world

<OpenAIObject at 0x7f3993f61490> JSON: {
  "content": "{\n  \"name\": \"Fantasia\",\n  \"regions\": [\n    {\n      \"name\": \"The Great Forest\",\n      \"description\": \"A vast woodland of ancient trees and hidden dangers.\",\n      \"pointsOfInterest\": [\n        {\n          \"name\": \"The Sentinel Tree\",\n          \"description\": \"An enormous tree rumored to have a conscious spirit that watches over the forest.\"\n        },\n        {\n          \"name\": \"The Crystal Caves\",\n          \"description\": \"A series of underground caves filled with glittering crystals and dangerous creatures.\"\n        }\n      ],\n      \"settlements\": [\n        {\n          \"name\": \"Greenhaven\",\n          \"description\": \"A small elven village built within the branches of the trees.\",\n          \"size\": \"VILLAGE\"\n        },\n        {\n          \"name\": \"Oakheart\",\n          \"description\": \"A large human city on the edge of the forest, known for its skilled woodw

In [42]:
from jsoncomment import JsonComment
json = JsonComment()

world_parsed = json.loads(world['content'])


In [68]:
print(f"World: {world_parsed['name']}")
for region in world_parsed['regions']:
    print(f"\tRegion: {region['name']}")
    for settlement in region['settlements']:
        print(f"\t\tSettlement: {settlement['name']} ({settlement['size']})")
        for faction in world_parsed['factions']:
            # print(f"\t\t\t{faction}")
            presence = [l['presence'] for l in faction['locations'] if l['settlement'] == settlement['name']]
            if len(presence) > 0:
                presence = presence[0]
                print(f"\t\t\tFaction: {faction['name']} ({presence})")
    pois = [p['name'] for p in region['pointsOfInterest']]
    pois = ', '.join(pois)
    print(f"\t\tPoints of Interest: {pois}")

World: Fantasia
	Region: The Great Forest
		Settlement: Greenhaven (VILLAGE)
			Faction: The Emerald Enclave (ESTABLISHED)
		Settlement: Oakheart (CITY)
		Settlement: Wildwood (TOWN)
			Faction: The Emerald Enclave (LIMITED)
		Points of Interest: The Sentinel Tree, The Crystal Caves
	Region: The Iron Mountains
		Settlement: Ironhold (CITY)
			Faction: The Iron Legion (ESTABLISHED)
			Faction: The Crimson Brotherhood (LIMITED)
		Settlement: Goldseeker's Rest (VILLAGE)
		Settlement: Skullcrusher's Keep (TOWN)
			Faction: The Iron Legion (LIMITED)
		Points of Interest: The Dwarven Stronghold, The Firefall Caverns
	Region: The Sunken Coast
		Settlement: Seagull's Landing (TOWN)
			Faction: The Gilded Court (ESTABLISHED)
		Settlement: Mariner's Rest (VILLAGE)
			Faction: The Gilded Court (LIMITED)
		Settlement: Siren's Call (VILLAGE)
		Points of Interest: The Mermaid's Lagoon, The Pirate's Cove
	Region: The Endless Wastes
		Settlement: Sandhaven (VILLAGE)
		Settlement: Oasis Springs (TOWN)


Generating the world all in one shot seems tough. May be nice to have a bit more hierarchy, and definitely want to incorporate some spatial aspects.

In [116]:
def continent_prompt(continents=(3, 3), regions=(2, 5), factions=(3, 6)): 
    return f'''
    Create the rough sketches of a fantasy world.
    
    JSON Format:

    sovereign_factions:
            name: ''
            race: ''
            description: ''
    continents:
        relative_location: ''
        regions:
            relative_location: ''
            biome_type: ''
            sovereign_faction: <valid reference to sovereign faction>
    
    The following constraints MUST be met.

    There must be at least {factions[0]} and no more than {factions[1]} distinct total sovereign factions. They should use standard playable races from 5e. The names should be interesting.
    
    There must be at least {continents[0]} and no more than {continents[1]} continents. There must be at least {regions[0]} and no more than {regions[1]} regions per continent. 
    Relative locations should include both cardinal directions and general position in the content (e.g. inland, coastal).

    Each region must have a valid sovereign faction. This implies that factions may control multiple regions. When this occurs, these regions of a faction's control should be adjacent.
    A faction cannot control a region on more than one continent.
    '''

In [117]:
continents_response = system_completion(continent_prompt())
continents = json.loads(continents_response['content'])
continents

{'sovereign_factions': [{'name': 'Celestial Dominion',
   'race': 'Aasimar',
   'description': 'Rulers of the sacred city in the clouds, worshipers of the divine.'},
  {'name': 'Draconic Empire',
   'race': 'Dragonborn',
   'description': 'Mighty empire ruled by powerful dragons, with the capital at the heart of a vast desert.'},
  {'name': 'Elven Confederacy',
   'race': 'Elf',
   'description': 'Alliance of various elven clans, known for their archery and their mastery of magic.'},
  {'name': 'Gilded Republic',
   'race': 'Human',
   'description': 'Merchant republic of powerful traders, with a monopoly on precious metals and gemstones.'},
  {'name': 'Orcish Hordes',
   'race': 'Orc',
   'description': 'Savage tribes of bloodthirsty warriors, united under the leadership of a powerful warlord.'},
  {'name': 'Undead Dominion',
   'race': 'Undead',
   'description': 'Necromancers who have found a way to prolong undeath and amass an army of undead minions.'}],
 'continents': [{'relative_